In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/customized-knowledge-qa
!pip install python-docx pyvi rank-bm25

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/customized-knowledge-qa


# **Preprocess text**

In [2]:
import re

def clean_text(text):
    text = re.sub('<.*?>', '', text).strip()
    text = re.sub('(\s)+', r'\1', text)
    return text

In [3]:
import string

def normalize_text(text):
    listpunctuation = string.punctuation.replace('_', '')
    for i in listpunctuation:
        text = text.replace(i, ' ')
    return text.lower()

In [4]:
import pandas as pd

filename = 'knowledge/stopwords.csv'
data = pd.read_csv(filename, sep="\t", encoding='utf-8')
list_stopwords = data['stopwords']

def remove_stopword(text):
    pre_text = []
    words = text.split()
    for word in words:
        if word not in list_stopwords:
            pre_text.append(word)
    text2 = ' '.join(pre_text)
    return text2

In [5]:
from pyvi.ViTokenizer import tokenize

def word_segment(sent):
    sent = tokenize(sent.encode('utf-8').decode('utf-8'))
    return sent

# **Prepare data**

In [ ]:
import os

data_dir = "knowledge/data"
documents = []
txt_files = [file for file in os.listdir(data_dir) if file.endswith(".txt")]

# Preprocess contents of .txt files
for txt_file in txt_files:
    file_path = os.path.join(data_dir, txt_file)
    with open(file_path, 'r', encoding='utf-8') as file:
        content = remove_stopword(normalize_text(word_segment(clean_text(file.read()))))
        documents.append(content)

# Create a list of tokenized documents
tokenized_documents = [doc.split() for doc in documents]

# **Okapi BM25**

In [15]:
from rank_bm25 import BM25Okapi

bm25 = BM25Okapi(tokenized_documents)

# **Example search**

In [16]:
# Question to search for relevant context
question = 'Nguyên nhân dẫn đến biên lợi nhuận gộp của C4G tại Q4/22 giảm mạnh ?'

# Tokenize the question
tokenized_question = question.split()

# Get the top N documents that are most relevant to the question
top_k = 3
doc_scores = bm25.get_scores(tokenized_question)
top_indices = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i], reverse=True)[:top_k]

# Print the relevant context for answering the question
for index in top_indices:
    print(f"Document {index + 1}: {documents[index]}")

Document 9: c4g – khẳng_định năng_lực trong đầu_tư công kqkd q4 22 tăng mạnh trên nền thấp 2021 ln cả năm theo sát dự phóng doanh_thu dt q4 22 tăng 79 5 svck lên 928 8 tỷ đồng do doanh_thu mảng xây_dựng của c4g tăng 58 2 svck tuy_nhiên biên ln gộp giảm mạnh 8 9 điểm svck do giá nguyên vật_liệu xây_dựng tăng mạnh đầu năm 2022 dẫn đến biên ln gộp của mảng xây_dựng giảm 6 9 điểm svck kết_quả ln ròng q4 22 tăng gấp 5 9 lần svck lên 58 3 tỷ đồng lũy kế cả năm 2022 ln ròng của c4g tăng 173 2 svck lên 168 1 tỷ đồng hoàn_thành 101 2 dự phóng của chúng_tôi nhiều tín_hiệu tích_cực từ giải_ngân đầu_tư công ngay từ đầu năm 2023 chính_phủ đã thể_hiện sự quyết_liệt trong việc giải_ngân đầu_tư công năm 2023 thể_hiện ở việc 1 quốc_hội đã thông_qua kế_hoạch đầu_tư công năm nay với tổng vốn gần 711 700 tỷ đồng tăng khoảng 22 3 so với kế_hoạch 2022 2 12 25 gói_thầu đầu_tiên tại cao_tốc bắc nam giai_đoạn 2 đã khởi_công 3 13 gói_thầu còn lại đã lựa_chọn xong nhà_thầu vào ngày 10 01 2023 và 2 13 gói_thầu đã